# Парсер CIAN.RU

Загрузка библиотек

In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import math
import urllib.request
from pathlib import Path
import os

import warnings
warnings.simplefilter('ignore')

Путь к рабочей папке

In [ ]:
path = Path().absolute()

Ниже собраны все регулярные выражения для поиска переменных в HTML-коде (для удобства обновления в случае их динамического изменения на стороне сервера)

In [ ]:
regex_dict = {
    'offersData':"(?<=\"offers\":\[)(.*)(?=\],\"aggregatedOffers\")",     
    'fullUrl':"(?<=fullUrl\":\")(.*?)(?=\",)",
    'cianId':"(?<=ru\\u002Fsale\\u002Fflat\\u002F)(.*?)(?=\\u002F\",)",
#     'photoUrl':"(?<=fullUrl\":\")(.*?)(?=\",)", # большой размер - 1
#     'photoUrl':"(?<=thumbnail2Url\":\")(.*?)(?=\",)", # средний размер - 4 
    'photoUrl':"(?<=thumbnailUrl\":\")(.*?)(?=\",)", # маленький размер - 2
    'totalOffers':"(?<=totalOffers\":)(.*?)(?=\,)",
    'description':"(?<=\"description\":\")(.*?)(?=\",)",
    'totalArea':"(?<=\"totalArea\":\")(.*?)(?=\",)",
    'loggiasCount':"(?<=loggiasCount\":)(.*?)(?=,)",
    'flatType':"(?<=flatType\":\")(.*?)(?=\",)",
    'kitchenArea':"(?<=kitchenArea\":\")(.*?)(?=\",)",
    'undergrounds':"(?<=undergrounds\":\[)(.*?)(?=])",
    'transport_time':"(?<=time\":)(.*?)(?=,)",
    'station_name':"(?<=name\":\")(.*?)(?=\")",
    'transportType':"(?<=transportType\":\")(.*?)(?=\")",
    'coord_lat':"(?<=lat\":)(.*?)(?=,)",
    'coord_lng':"(?<=lng\":)(.*?)(?=,)",
    'location':"(?<=location\",\"name\":\")(.*?)(?=\",)",
    'okrug':"(?<=okrug\",\"name\":\")(.*?)(?=\",)",
    'raion':"(?<=raion\",\"name\":\")(.*?)(?=\",)",
    'street':"(?<=street\",\"name\":\")(.*?)(?=\",)",
    'house':"(?<=house\",\"name\":\")(.*?)(?=\",)",
    'isApartments':"(?<=isApartments\":)(.*?)(?=,)",
    'creationDate':"(?<=creationDate\":\")(.*?)(?=\")",
    'status':"(?<=status\":\")(.*?)(?=\")",
    'floorNumber':"(?<=\"floorNumber\":)(.*?)(?=,)",
    'addedTimestamp':"(?<=\"addedTimestamp\":)(.*?)(?=,)",
    'livingArea':"(?<=livingArea\":\")(.*?)(?=\")",
    'materialType':"(?<=materialType\":\")(.*?)(?=\")",
    'buildYear':"(?<=\"buildYear\":)(.*?)(?=,)",
    'passengerLiftsCount':"(?<=\"passengerLiftsCount\":)(.*?)(?=,)",
    'floorsCount':"(?<=\"floorsCount\":)(.*?)(?=,)",
    'year':"(?<=\"year\":)(.*?)(?=,)",
    'isComplete':"(?<=\"isComplete\":)(.*?)(?=,)",
    'cargoLiftsCount':"(?<=\"cargoLiftsCount\":)(.*?)(?=,)",
    'roomsCount':"(?<=\"roomsCount\":)(.*?)(?=,)",
    'price':"(?<=\"price\":)(.*?)(?=,)",
    'mortgageAllowed':"(?<=\"mortgageAllowed\":)(.*?)(?=,)",
    'currency':"(?<=\"currency\":\")(.*?)(?=\")",
    'accountType':"(?<=\"accountType\":\")(.*?)(?=\")",
    'agencyName':"(?<=\"agencyName\":\")(.*?)(?=\")",
    'cianProfileStatus':"(?<=\"cianProfileStatus\":\")(.*?)(?=\")",
    'cianUserId':"(?<=\"cianUserId\":\")(.*?)(?=\")",
    'companyName':"(?<=\"companyName\":\")(.*?)(?=\")",
    'isAgent':"(?<=\"isAgent\":)(.*?)(?=,)",
    'isBuilder':"(?<=\"isBuilder\":)(.*?)(?=,)",
    'isPassportVerified':"(?<=\"isPassportVerified\":)(.*?)(?=,)",
    'isSelfEmployed':"(?<=\"isSelfEmployed\":)(.*?)(?=,)",
    'isSubAgent':"(?<=\"isSubAgent\":)(.*?)(?=,)",
    'userTrustLevel':"(?<=\"userTrustLevel\":\")(.*?)(?=\")",
    'userId':"(?<=\"userId\":)(.*?)(?=,)",
    'userType':"(?<=\"userType\":\")(.*?)(?=\")",
    'isCianPartner':"(?<=\"isCianPartner\":)(.*?)(?=,)",
    'userInput':"(?<=\"userInput\":\")(.*?)(?=\")",
}

In [ ]:
districts_dict = {
#     'Новомосковский административный округ':325, #округ
        'Внуковское':328, 'Воскресенское':327, 'Десёновское':329, 'Кокошкино':330, 'Марушкинское':238, 'Московский':332,        'Мосрентген':333,        'Рязановское':334,
        'Сосенское':335, 'Филимонковское':336, 'Щербинка':337,
#     'Троицкий административный округ':326, #округ 
        'Вороновское':338, 'Киевский':339, 'Клёновское':340, 'Краснопахорское':341, 'Михайлово-Ярцевское':342,
        'Новофёдоровское':343, 'Первомайское':344, 'Роговское':345, 'Троицк':346, 'Щаповское':347,
#     'Зеленоградский административный округ':151, #округ
        'Крюково':154, 'Матушкино':355, 'Матушкино-Савёлки':152, 'Панфиловский':153, 'Савёлки':356, 'Силино':357,
        'Старое Крюково':358,
#     'Северо-Западный административный округ':1, #округ
        'Куркино':125, 'Митино':126, 'Покровское-Стрешнево':127, 'Северное Тушино':128, 'Строгино':129,
        'Хорошево-Мневники':130, 'Щукино':131, 'Южное Тушино':132,
#     'Западный административный округ':11, #округ
        'Внуково':112, 'Дорогомилово':113, 'Крылатское':114, 'Кунцево':115, 'Можайский':116, 'Ново-Переделкино':117,
        'Очаково-Матвеевское':118, 'Проспект Вернадского':119, 'Раменки':120, 'Солнцево':121, 'Тропарево-Никулино':122,
        'Филевский парк':123, 'Фили-Давыдково':124, 'Конезавод':349, 'Рублёво-Архангельское':348, 'Сколково':350,
#     'Юго-Западный административный округ':10, #округ
        'Академический':100, 'Гагаринский':101, 'Зюзино':102, 'Коньково':103, 'Котловка':104, 'Ломоносовский':105,
        'Обручевский':106, 'Северное Бутово':107, 'Теплый Стан':108, 'Черемушки':109, 'Южное Бутово':110, 'Ясенево':111,
#     'Северный административный округ':5, #округ
        'Аэропорт':23, 'Беговой':24, 'Бескудниковский':25, 'Войковский':26, 'Восточное Дегунино':27, 'Головинский':28,
        'Дмитровский':29, 'Западное Дегунино':30, 'Коптево':31, 'Левобережный':32, 'Молжаниновский':33, 'Савеловский':34,
        'Сокол':35, 'Тимирязевский':36, 'Ховрино':37, 'Хорошевский':38, 
#     'Центральный административный округ':4, #округ
        'Арбат':13, 'Басманный':14, 'Замоскворечье':15, 'Красносельский':16, 'Мещанский':17, 'Пресненский':18, 
        'Таганский':19, 'Тверской':20, 'Хамовники':21, 'Якиманка':22, 
#     'Южный административный округ':9, #округ
        'Бирюлево Восточное':84, 'Бирюлево Западное':85, 'Братеево':86, 'Даниловский':87, 'Донской':88, 'Зябликово':89, 
        'Москворечье-Сабурово':90, 'Нагатино-Садовники':91, 'Нагатинский затон':92, 'Нагорный':93, 
        'Орехово-Борисово Северное':94, 'Орехово-Борисово Южное':95, 'Царицыно':96, 'Чертаново Северное':97, 
        'Чертаново Центральное':98, 'Чертаново Южное':99, 
#     'Северо-Восточный административный округ':6, #округ
        'Алексеевский':39, 'Алтуфьевский':40, 'Бабушкинский':41, 'Бибирево':42, 'Бутырский':43, 'Лианозово':44, 
        'Лосиноостровский':45, 'Марфино':46, 'Марьина роща':47, 'Останкинский':48, 'Отрадное':49, 'Ростокино':50, 
        'Свиблово':51, 'Северное Медведково':53, 'Северный':52, 'Южное Медведково':54, 'Ярославский':55, 
#     'Восточный административный округ':7, #округ
        'Богородское':56, 'Вешняки':57, 'Восточное Измайлово':59, 'Восточный':58, 'Гольяново':60, 'Ивановское':61,
        'Измайлово':62, 'Косино-Ухтомский':63, 'Метрогородок':64, 'Новогиреево':65, 'Новокосино':66, 'Перово':67, 
        'Преображенское':68, 'Северное Измайлово':69, 'Соколиная гора':70, 'Сокольники':71, 
#     'Юго-Восточный административный округ':8, #округ
        'Выхино-Жулебино':72, 'Капотня':73, 'Кузьминки':74, 'Лефортово':75, 'Люблино':76, 'Марьино':77, 'Некрасовка':78, 
        'Нижегородский':79, 'Печатники':80, 'Рязанский':81, 'Текстильщики':82, 'Южнопортовый':83,
}

Переменные, используемые в классе CianParser

In [25]:
file_districts = str(path) + '/df_districts.csv'
file_flats = str(path) + '/df_flats.csv'
dir_photo = 'D://cian_photo/'
# dir_photo = 'C://cian_photo/'
district_columns = ['id', 'district_name', 'district_type']

flat_columns = [
    'id',
    'url',
    'timestamp_parsed',
    'description',
    'total_area',
    'loggias_count',
    'flat_type',
    'kitchen_area',
    'station_name1',
    'station_name2',
    'station_name3',
    'transport_type1',
    'transport_type2',
    'transport_type3',
    'transport_time1',
    'transport_time2',
    'transport_time3',
    'coord_lat',
    'coord_lng',
    'location',
    'okrug',
    'raion',
    'street',
    'house',
    'is_apartments',
    'creation_date',
    'status',
    'floor_number',
    'added_timestamp',
    'living_area',
    'material_type',
    'build_year',
    'passenger_lifts_count',
    'floors_count',
    'year',
    'is_complete',
    'cargo_lifts_count',
    'rooms_count',
    'price',
    'mortgage_allowed',
    'currency',
    'account_type',
    'agency_name',
    'cian_profile_status',
    'cian_user_id',
    'company_name',
    'is_agent',
    'is_builder',
    'is_passport_verified',
    'is_self_employed',
    'is_sub_agent',
    'user_trust_level',
    'user_id',
    'user_type',
    'is_cian_partner',
    'user_address',
]

In [26]:
if not os.path.exists(dir_photo):
    os.mkdir(dir_photo)

Пример полной ссылки по всем квартирам для одного конкретного района:

<a href="https://www.cian.ru/cat.php?deal_type=sale&district[0]=112&engine_version=2&offer_type=flat&p=1&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room8=1&room9=1">Квартиры в районе Внуково</a>

https://www.cian.ru/cat.php?deal_type=sale&district[0]=112&engine_version=2&offer_type=flat&p=1&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room8=1&room9=1

In [27]:
class CianParser(): 
    """
    Класс парсинга данных из CIAN в pandas-датасет    
                 regex_dict : словарь регулярных выражений для поиска переменных в HTML-коде
                 districts : список районов, для которых парсятся предложения 
                 verbose : булевый признак отображения логов
                 wait : целочисленное значение задержки между переходом на следующую страницу, в секундах
                         требуется для того, чтобы не попасть на капчу
                 file_districts : файл датасета с данными по районам
                 file_flats : файл датасета с данными по квартирам (основной датасет)
                 dir_photo : папка с фотографиями квартир
                 district_columns : столбцы датасета с данными по районам
                 flat_columns : столбцы датасета с данными по квартирам
    """
    
    # init
    def __init__(self,
                 regex_dict=regex_dict,
                 districts=None, 
                 verbose=True,
                 wait=10,
                 file_districts=file_districts,
                 file_flats = file_flats,
                 dir_photo = dir_photo,
                 district_columns = district_columns,
                 flat_columns = flat_columns,
                ):
        self.regex_dict = regex_dict
        self.districts = districts
        self.verbose = verbose
        self.wait = wait
        self.file_districts = file_districts
        self.file_flats = file_flats
        self.dir_photo = dir_photo
        self.district_columns = district_columns
        self.flat_columns = flat_columns
        
        self.dir_district = None
                
    # генератор основной ссылки на список квартир в районе
    # на вход подается номер страницы, номер района, 9 признаков по количеству комнат
    # возвращает текстовую переменную ссылки
    def get_page(self, pagenum, district_id, room1, room2, room3, room4, room5, room6, room7, room8, room9):
        page = 'https://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=' + str(district_id) + \
                '&engine_version=2&offer_type=flat&p=' + str(pagenum) + \
                '&region=1&room1=' + str(room1) + \
                '&room2=' + str(room2) + \
                '&room3=' + str(room3) + \
                '&room4=' + str(room4) + \
                '&room5=' + str(room5) + \
                '&room6=' + str(room6) + \
                '&room7=' + str(room7) + \
                '&room8=' + str(room8) + \
                '&room9=' + str(room9)
        return page
    
    # загрузчик HTML-содержимого
    # на вход подается ссылка на страницу
    # возвращает BS-переменную с HTML-содержимым страницы
    def get_soup(self, page):
        time.sleep(self.wait)
        soup = None
        redirect_1page = 'offer_type=flat&p=1' 
        response = requests.get(page, allow_redirects=False)
        if response.status_code != 200:
            if (redirect_1page in response.url) & (redirect_1page not in page):
                if self.verbose:
                    print(f' {response.status_code}', end='')
                soup = None
            elif 'captcha' in response.url:
                if self.verbose:
                    print('c', end='')
                time.sleep(360)
                response = requests.get(page, allow_redirects=False)
            else: 
                for i in range(10):
                    time.sleep(self.wait * 10)
                    response = requests.get(page, allow_redirects=False)
                    if response.status_code == 200:
                        break
        if response.status_code == 200:
            html = response.content
            soup = BeautifulSoup(html,'html.parser')
        return soup

    # загрузчик датасета
    # на вход подается название файла, из которого он будет загружен, 
    # и список его столбцов, в случае если датасет еще не создан
    # возвращает датасет и список содержимого колонки id в нем
    def load_df(self, my_file, columns):
        my_path = Path(my_file)
        if my_path.is_file():
            df = pd.read_csv(my_file)
            ids = list(df.id)
        else:
            df = pd.DataFrame(columns=columns)
            ids = []
        return df, ids

    # сохраняет датасет
    # на вход подается датасет и название файла, в который он будет сохранен
    # ничего не возвращает
    def save_df(self, df, my_file):
        if df.shape[0] > 0:
            df.to_csv(my_file, index=False)
        else:
            if self.verbose:
                print('No rows in df!')  
       
    # на вход подается js-дамп текста по конкретному объявлению, id объявления
    # ничего не возвращает, но ищет и сохраняет в отдельную папку все фотографии в данном объявлении     
    def get_photo(self, data, cianId):
        matches = self.get_matches(data, self.regex_dict['photoUrl'])
        matches = [item for item in matches if item.endswith('-2.jpg')]
        if matches:
            dir_flat = f'{self.dir_district}/{cianId}'
            if not os.path.exists(dir_flat):
                 os.mkdir(dir_flat)
            for i, url in zip(range(len(matches)), matches):
                url = url.replace("\\u002F", '/')
#                 urllib.request.urlretrieve(url, str(f"{dir_flat_photo}\\{cianId}_{i}.jpg"))
                try:
                    urllib.request.urlretrieve(url, str(f"{dir_flat}/{cianId}_{i}.jpg"))
                except:
                    print('p', end='')

    # основной метод, вызываемый для поиска и парсинга объявлений
    # на вход подается 9 признаков по количеству комнат
    # ничего не возвращает   
    def main(self, room1=0, room2=0, room3=0, room4=0, room5=0, room6=0, room7=0, room8=0, room9=0):
        # логирование
        if self.verbose:
            print('ПАРСЕР КВАРТИР С САЙТА CIAN.RU')
            print(f'Интервал между запросами: {self.wait} сек.')  
        
        # поиск квартир по всем районам
        for district_id in self.districts: 
            self.parsing(district_id, room1, room2, room3, room4, room5, room6, room7, room8, room9)

            
    # метод, вызываемый для js-дампа по регулярному выражению
    # на вход подается BS-содержимое и регулярное выражение
    # возвращает текст js-дампа со всеми js-переменными по объявлениям на странице
    def get_scriptData(self, soup, regex):
        script = None
        pattern = re.compile(r"%s" % regex)
        script = soup.find("script", text=pattern)
        if script:
            script = script.text.replace('{', '').replace('}', '')
        return script

    # на вход подается js-дамп и регулярное выражение
    # возвращает id предложения  по регулярному выражению
    def get_id(self, data):
        match = None
        pattern = re.compile(r"(?<=ru\\u002Fsale\\u002Fflat\\u002F)(.*?)(?=\\u002F\",)")
        match = re.search(pattern, data)
        if match:
            match = match.group(1)
        return match
    
    # метод, который находит 1 матч по регулярному выражению
    # на вход подается js-дамп и регулярное выражение
    # возвращает 1 матч  по регулярному выражению
    def get_match(self, data, regex):
        match = None
        pattern = re.compile(r"%s" % regex)
        match = re.search(pattern, data)
        if match:
            match = match.group(1)
        return match
    
    # метод, который находит все матчи по регулярному выражению
    # на вход подается js-дамп и регулярное выражение
    # возвращает список матчей по регулярному выражению
    def get_matches(self, data, regex):
        matches = None
        pattern = re.compile(r"%s" % regex)
        matches = re.findall(pattern, data)
        return matches
    
    # метод, который находит все js-дампы объявлений
    # на вход подается js-дамп
    # возвращает список js-дампов объявлений
    def get_offersData(self, data):
        offersData = None
        match = self.get_match(data, self.regex_dict['offersData'])
        if match:
            offersData = match.split('siteBlockId')[:-1]
        return offersData
    
    # находит три ближайших станции метро
    # возвращает списки с названиями станций, временем в пути до них, типом передвижения
    def get_underground(self, soup):
        station_name = [None, None, None]
        transportType = [None, None, None]
        transport_time = [None, None, None]

        match = self.get_match(soup, self.regex_dict['undergrounds'])
        station_names = self.get_matches(match, self.regex_dict['station_name'])
        transportTypes = self.get_matches(match, self.regex_dict['transportType'])
        transport_times = self.get_matches(match, self.regex_dict['transport_time'])
        if station_names:
            for i in range(len(station_names)):
                station_name[i] = station_names[i]
        if transportTypes:
            for i in range(len(transportTypes)):
                transportType[i] = transportTypes[i]
        if transport_times:
            for i in range(len(transport_times)):
                transport_time[i] = transport_times[i]

        return station_name[0], station_name[1], station_name[2], \
                transportType[0], transportType[1], transportType[2], \
                transport_time[0], transport_time[1], transport_time[2]
        
    # парсер страниц
    # на вход подается номер страницы, номер района, 9 признаков по количеству комнат
    # ничего не возвращает, но сохраняет все найденные объявления в датасет с данными по квартирам
    def parsing(self, district_id, room1, room2, room3, room4, room5, room6, room7, room8, room9):
        
        OFFERS_ON_PAGE = 28
        self.dir_district = f'{self.dir_photo}/{district_id}'
        if not os.path.exists(self.dir_district):
            os.mkdir(self.dir_district)
            
        page1 = self.get_page(1, district_id, room1, room2, room3, room4, room5, room6, room7, room8, room9)
        soup1 = self.get_soup(page1)
        if soup1:
            script = self.get_scriptData(soup1, self.regex_dict['offersData'])
            totalOffers = self.get_match(script, self.regex_dict['totalOffers'])
        else:
            totalOffers = None
        
        df, ids = self.load_df(self.file_flats, self.flat_columns)
        if totalOffers:
            maxpage = math.ceil(float(totalOffers) / OFFERS_ON_PAGE) + 1
            # логирование
            if self.verbose:
                print(f'Dstr:{district_id} |Offr:{totalOffers} |Pgs:{maxpage-1} [', end='')    
            
            newflat_cnt = 0
            for i in range(1, maxpage):
                # логирование
                if self.verbose:
                    print('.', end='')
                page = self.get_page(i, district_id, room1, room2, room3, room4, room5, room6, room7, room8, room9)
                soup = self.get_soup(page)
                if soup:
                    script = self.get_scriptData(soup, self.regex_dict['offersData'])
                    offersData = self.get_offersData(script)   
                    if offersData:
                        for data in offersData:
                            curr_id = self.get_id(data)
                            if curr_id:
                                if int(curr_id) not in ids:
                                    newflat_cnt += 1
                                    data_dict = {}
                                    data_dict['id'] = curr_id
                                    data_dict['url'] = f'https://www.cian.ru/sale/flat/{curr_id}/'
                                    data_dict['timestamp_parsed'] = time.time()

                                    self.get_photo(data, data_dict['id'])

                                    data_dict['description'] = self.get_match(data, self.regex_dict['description'])
                                    data_dict['total_area'] = self.get_match(data, self.regex_dict['totalArea'])
                                    data_dict['loggias_count'] = self.get_match(data, self.regex_dict['loggiasCount'])
                                    data_dict['flat_type'] = self.get_match(data, self.regex_dict['flatType'])
                                    data_dict['kitchen_area'] = self.get_match(data, self.regex_dict['kitchenArea'])
                                    data_dict['station_name1'], data_dict['station_name2'], data_dict['station_name3'], \
                                    data_dict['transport_type1'], data_dict['transport_type2'], data_dict['transport_type3'], \
                                    data_dict['transport_time1'], data_dict['transport_time2'], data_dict['transport_time3']  \
                                    = self.get_underground(data)
                                    data_dict['coord_lat'] = self.get_match(data, self.regex_dict['coord_lat'])
                                    data_dict['coord_lng'] = self.get_match(data, self.regex_dict['coord_lng'])
                                    data_dict['location'] = self.get_match(data, self.regex_dict['location'])
                                    data_dict['okrug'] = self.get_match(data, self.regex_dict['okrug'])
                                    data_dict['raion'] = self.get_match(data, self.regex_dict['raion'])
                                    data_dict['street'] = self.get_match(data, self.regex_dict['street'])
                                    data_dict['house'] = self.get_match(data, self.regex_dict['house'])
                                    data_dict['is_apartments'] = self.get_match(data, self.regex_dict['isApartments'])
                                    data_dict['creation_date'] = self.get_match(data, self.regex_dict['creationDate'])
                                    data_dict['status'] = self.get_match(data, self.regex_dict['status'])
                                    data_dict['floor_number'] = self.get_match(data, self.regex_dict['floorNumber'])
                                    data_dict['added_timestamp'] = self.get_match(data, self.regex_dict['addedTimestamp'])
                                    data_dict['living_area'] = self.get_match(data, self.regex_dict['livingArea'])
                                    data_dict['material_type'] = self.get_match(data, self.regex_dict['materialType'])
                                    data_dict['build_year'] = self.get_match(data, self.regex_dict['buildYear'])
                                    data_dict['passenger_lifts_count'] = self.get_match(data, self.regex_dict['passengerLiftsCount'])
                                    data_dict['floors_count'] = self.get_match(data, self.regex_dict['floorsCount'])
                                    data_dict['year'] = self.get_match(data, self.regex_dict['year'])
                                    data_dict['is_complete'] = self.get_match(data, self.regex_dict['isComplete'])                           
                                    data_dict['cargo_lifts_count'] = self.get_match(data, self.regex_dict['cargoLiftsCount'])
                                    data_dict['rooms_count'] = self.get_match(data, self.regex_dict['roomsCount'])
                                    data_dict['price'] = self.get_match(data, self.regex_dict['price'])
                                    data_dict['mortgage_allowed'] = self.get_match(data, self.regex_dict['mortgageAllowed'])
                                    data_dict['currency'] = self.get_match(data, self.regex_dict['currency'])
                                    data_dict['account_type'] = self.get_match(data, self.regex_dict['accountType'])
                                    data_dict['agency_name'] = self.get_match(data, self.regex_dict['agencyName'])
                                    data_dict['cian_profile_status'] = self.get_match(data, self.regex_dict['cianProfileStatus'])
                                    data_dict['cian_user_id'] = self.get_match(data, self.regex_dict['cianUserId'])
                                    data_dict['company_name'] = self.get_match(data, self.regex_dict['companyName'])
                                    data_dict['is_agent'] = self.get_match(data, self.regex_dict['isAgent'])
                                    data_dict['is_builder'] = self.get_match(data, self.regex_dict['isBuilder'])
                                    data_dict['is_passport_verified'] = self.get_match(data, self.regex_dict['isPassportVerified'])
                                    data_dict['is_self_employed'] = self.get_match(data, self.regex_dict['isSelfEmployed'])
                                    data_dict['is_sub_agent'] = self.get_match(data, self.regex_dict['isSubAgent'])
                                    data_dict['user_trust_level'] = self.get_match(data, self.regex_dict['userTrustLevel'])
                                    data_dict['user_id'] = self.get_match(data, self.regex_dict['userId'])
                                    data_dict['user_type'] = self.get_match(data, self.regex_dict['userType'])
                                    data_dict['is_cian_partner'] = self.get_match(data, self.regex_dict['isCianPartner'])
                                    data_dict['user_address'] = self.get_match(data, self.regex_dict['userInput'])
                                    
                                    df = df.append(pd.Series(data_dict), ignore_index=True)
                                    ids.append(int(curr_id))
#                         self.save_df(df, self.file_flats)
                else:
                    break
                
            # логирование
            if self.verbose:
                print(f'] {newflat_cnt} found!')
        else:
            print(f'No offers in {district_id}!')
        self.save_df(df, self.file_flats)    


Экземпляр класса CianParser

In [28]:
cian_parser = CianParser(regex_dict=regex_dict, wait=3, districts=list(districts_dict.values()))
# cian_parser = CianParser(regex_dict=regex_dict, wait=3, districts=[13])

Поиск и парсинг объявлений

In [29]:
cian_parser.main(room1=1, room2=1, room3=1, room4=1, room5=1, room6=1, room7=1, room8=1, room9=1)

ПАРСЕР КВАРТИР С САЙТА CIAN.RU
Интервал между запросами: 3 сек.
Dstr:328 |Offr:899 |Pgs:33 [...............p............] 448 found!
Dstr:327 |Offr:134 |Pgs:5 [.....] 117 found!
Dstr:329 |Offr:868 |Pgs:31 [...........................] 264 found!
Dstr:330 |Offr:43 |Pgs:2 [..] 40 found!
Dstr:238 |Offr:399 |Pgs:15 [........] 195 found!
Dstr:332 |Offr:2086 |Pgs:75 [.........] 118 found!
Dstr:333 |Offr:33 |Pgs:2 [..] 32 found!
Dstr:334 |Offr:923 |Pgs:33 [...........................] 216 found!
Dstr:335 |Offr:4843 |Pgs:173 [.......ppppppppppppppppppp.p..........] 285 found!
No offers in 336!
Dstr:337 |Offr:233 |Pgs:9 [.........] 180 found!
Dstr:338 |Offr:5 |Pgs:1 [.] 0 found!
Dstr:339 |Offr:12 |Pgs:1 [.] 1 found!
Dstr:340 |Offr:22 |Pgs:1 [.] 1 found!
Dstr:341 |Offr:18 |Pgs:1 [.] 1 found!
Dstr:342 |Offr:9 |Pgs:1 [.] 1 found!
Dstr:343 |Offr:148 |Pgs:6 [......] 3 found!
Dstr:344 |Offr:145 |Pgs:6 [......] 18 found!
Dstr:345 |Offr:31 |Pgs:2 [..] 3 found!
Dstr:346 |Offr:128 |Pgs:5 [.....] 12 found